In [2]:
### Load library -----
library(DESeq2)
library(tidyverse)


In [3]:
### import GO DB -----
#Import GO database
GOdata<-read.csv('Reference csv/Cleaned_GO_4.csv', sep=';', header=FALSE)
GOdata$Gene<-as.character(GOdata$V1)
GOdata$V1<-NULL
GOdata$geneabb<-as.character(GOdata$V2)
GOdata$V2<-NULL
GOdata$genename<-as.character(GOdata$V3)
GOdata$V3<-NULL
GOdata$GO<-as.character(GOdata$V4)
GOdata$V4<-NULL

### featureCount data import -----
#Importing output of featurecounts
countdata <- read.table("FeatureCounts/Maya2_Root", header=TRUE, row.names='Geneid' )

#remove first 5 lines which is useless
countdata <- countdata[ ,6:ncol(countdata)]

#remove .bam in the sample name
colnames(countdata) <- gsub("\\_a.bam$", "", colnames(countdata))
rownames(countdata) <- gsub("gene:", "", rownames(countdata))
colnames(countdata)

#Import sample information
saminfo <- read.csv("Saminfo/Maya2_root.csv")
saminfo
#change column names using saminfo
colnames(countdata) <- saminfo$condition
colnames(countdata)

#sort by column names
saminfo <- arrange(saminfo, index)
countdata <- countdata[,c(saminfo$index)]

#change it to matrix
countdata <- as.matrix(countdata)

head(countdata)


[1] "X183421_34" "X183422_35" "X183423_36" "X183424_37" "X183425_38"
 [6] "X183426_39" "X183427_40" "X183428_41" "X183429_42" "X183430_43"
[11] "X183431_44" "X183432_45" "X183433_46" "X183434_47" "X183435_48"
[16] "X183436_49" "X183437_50" "X183438_51" "X183439_52" "X183440_53"
[21] "X183441_54" "X183442_55"

sample,condition,index
<int>,<chr>,<int>
183421,WT_0h_Maya2,1
183422,WT_0h_Maya2,2
183423,WT_0h_Maya2,3
183424,WT_1h_Maya2,4
183425,WT_1h_Maya2,5
183426,WT_1h_Maya2,6
183427,WT_2h_Maya2,7
183428,WT_2h_Maya2,8
183429,WT_2h_Maya2,9


[1] "WT_0h_Maya2"   "WT_0h_Maya2"   "WT_0h_Maya2"   "WT_1h_Maya2"  
 [5] "WT_1h_Maya2"   "WT_1h_Maya2"   "WT_2h_Maya2"   "WT_2h_Maya2"  
 [9] "WT_2h_Maya2"   "fls2_0h_Maya2" "fls2_0h_Maya2" "fls2_0h_Maya2"
[13] "fls2_1h_Maya2" "fls2_1h_Maya2" "fls2_1h_Maya2" "fls2_2h_Maya2"
[17] "fls2_2h_Maya2" "fls2_2h_Maya2" "WT_1h_flg22"   "WT_1h_flg22"  
[21] "WT_2h_flg22"   "WT_2h_flg22"

,WT_0h_Maya2,WT_0h_Maya2.1,WT_0h_Maya2.2,WT_1h_Maya2,WT_1h_Maya2.1,WT_1h_Maya2.2,WT_2h_Maya2,WT_2h_Maya2.1,WT_2h_Maya2.2,fls2_0h_Maya2,⋯,fls2_1h_Maya2,fls2_1h_Maya2.1,fls2_1h_Maya2.2,fls2_2h_Maya2,fls2_2h_Maya2.1,fls2_2h_Maya2.2,WT_1h_flg22,WT_1h_flg22.1,WT_2h_flg22,WT_2h_flg22.1
AT1G01010,168,68,92,65,90,94,115,141,172,80,⋯,98,92,213,203,137,219,192,155,156,191
AT1G01020,246,160,140,161,127,183,143,253,225,168,⋯,135,148,255,271,161,244,248,160,349,255
AT1G03987,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AT1G01030,19,3,22,13,13,18,13,15,20,7,⋯,18,20,36,20,18,37,19,10,45,33
AT1G01040,287,148,123,160,134,174,135,275,258,223,⋯,142,202,399,423,361,444,375,249,689,304
AT1G03993,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [4]:

### DEseq analysis -----
#define factor for DEseq
condition <- factor(c(rep('WT_0h_Maya2', 3), rep('WT_1h_Maya2', 3), rep('WT_2h_Maya2', 3),
                      rep('fls2_0h_Maya2', 3), rep('fls2_1h_Maya2', 3), rep('fls2_2h_Maya2', 3),
                      rep('WT_1h_flg22', 2), rep('WT_2h_flg22', 2)))

saminfo$colnames_count <- colnames(countdata)
coldata <- data.frame(row.names=colnames(countdata), condition)

#Chem screening
dds <- DESeqDataSetFromMatrix(countData=countdata, colData=coldata, design=~condition)
dds <- DESeq(dds)
colnames(dds) <- condition

res_wt <- results(dds, contrast=c('condition', 'WT_1h_Maya2', 'WT_0h_Maya2'))
res_fls2 <- results(dds, contrast=c('condition', 'fls2_1h_Maya2', 'fls2_0h_Maya2'))
res_flg22 <- results(dds, contrast=c('condition', 'WT_1h_flg22', 'WT_0h_Maya2'))

table(res_wt$padj<0.1)
table(res_fls2$padj<0.1)
table(res_flg22$padj<0.1)

resDF_wt<-as.data.frame(res_wt)
resDF_fls2<-as.data.frame(res_fls2)
resDF_flg22<-as.data.frame(res_flg22)

output_wt <- merge(resDF_wt, as.data.frame(counts(dds, normalized=TRUE)), by="row.names", sort=FALSE)
output_fls2 <- merge(resDF_fls2, as.data.frame(counts(dds, normalized=TRUE)), by="row.names", sort=FALSE)
output_flg22 <- merge(resDF_flg22, as.data.frame(counts(dds, normalized=TRUE)), by="row.names", sort=FALSE)

names(output_wt)[1] <- "Gene"
names(output_fls2)[1] <- "Gene"
names(output_flg22)[1] <- "Gene"

output_wt<-merge(GOdata, output_wt, by="Gene")
output_fls2<-merge(GOdata, output_fls2, by="Gene")
output_flg22<-merge(GOdata, output_flg22, by="Gene")

write.csv(output_wt, file="DESeq2/original/DEseq2_results_WT_Maya2_root.csv")
write.csv(output_fls2, file="DESeq2/original/DEseq2_results_fls2_Maya2_root.csv")
write.csv(output_flg22, file="DESeq2/original/DEseq2_results_flg22_root.csv")


estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing




FALSE  TRUE 
17680   829 


FALSE  TRUE 
18097   921 


FALSE  TRUE 
15856  3162 

In [5]:
### Generate final table -----
Total <- output_wt
Total$fls2.log2FoldChange <- output_fls2$log2FoldChange
Total$fls2.padj <- output_fls2$padj

colnames(Total)
Total <- Total %>%
  dplyr::select(Gene, geneabb, genename, log2FoldChange, padj,
                fls2.log2FoldChange, fls2.padj,
                starts_with("WT_0h_Maya2"), starts_with("WT_1h_Maya2"),
                starts_with("fls2_0h_Maya2"), starts_with("fls2_1h_Maya2"),
                GO)

colnames(Total) <- c("Gene", "Gene_Symbol", "Gene_Name", 
                     "Log2FC_WT", "Padj_WT", 
                     "Log2FC_fls2", "Padj_fls2", 
                     "WT_0h_Maya2_rep1", "WT_0h_Maya2_rep2", "WT_0h_Maya2_rep3",
                     "WT_1h_Maya2_rep1", "WT_1h_Maya2_rep2", "WT_1h_Maya2_rep3",                     
                     "fls2_0h_Maya2_rep1", "fls2_0h_Maya2_rep2", "fls2_0h_Maya2_rep3", 
                     "fls2_1h_Maya2_rep1", "fls2_1h_Maya2_rep2", "fls2_1h_Maya2_rep3", 
                     "GO")

write.csv(Total, file="Total/original/Total_Maya2_root.csv")

Total_with_flg22 <- output_wt
Total_with_flg22$fls2.log2FoldChange <- output_fls2$log2FoldChange
Total_with_flg22$fls2.padj <- output_fls2$padj
Total_with_flg22$flg22.log2FoldChange <- output_flg22$log2FoldChange
Total_with_flg22$flg22.padj <- output_flg22$padj
Total_with_flg22 <- Total_with_flg22 %>%
  dplyr::select(Gene, geneabb, genename, log2FoldChange, padj,
                fls2.log2FoldChange, fls2.padj,
                flg22.log2FoldChange, flg22.padj,
                starts_with("WT_0h_Maya2"), starts_with("WT_1h_Maya2"),
                starts_with("fls2_0h_Maya2"), starts_with("fls2_1h_Maya2"),
                starts_with("WT_1h_flg22"),
                GO)
colnames(Total_with_flg22) <- c("Gene", "Gene_Symbol", "Gene_Name", 
                     "Log2FC_WT", "Padj_WT", 
                     "Log2FC_fls2", "Padj_fls2",
                     "Log2FC_flg22", "Padj_flg22",
                     "WT_0h_Maya2_rep1", "WT_0h_Maya2_rep2", "WT_0h_Maya2_rep3",
                     "WT_1h_Maya2_rep1", "WT_1h_Maya2_rep2", "WT_1h_Maya2_rep3",                     
                     "fls2_0h_Maya2_rep1", "fls2_0h_Maya2_rep2", "fls2_0h_Maya2_rep3", 
                     "fls2_1h_Maya2_rep1", "fls2_1h_Maya2_rep2", "fls2_1h_Maya2_rep3",
                     "WT_1h_flg22_rep1","WT_1h_flg22_rep2",
                     "GO")

write.csv(Total_with_flg22, file="Total/original/Total_Maya2_root_with_flg22.csv")




[1] "Gene"                "geneabb"             "genename"           
 [4] "GO"                  "baseMean"            "log2FoldChange"     
 [7] "lfcSE"               "stat"                "pvalue"             
[10] "padj"                "WT_0h_Maya2"         "WT_0h_Maya2.1"      
[13] "WT_0h_Maya2.2"       "WT_1h_Maya2"         "WT_1h_Maya2.1"      
[16] "WT_1h_Maya2.2"       "WT_2h_Maya2"         "WT_2h_Maya2.1"      
[19] "WT_2h_Maya2.2"       "fls2_0h_Maya2"       "fls2_0h_Maya2.1"    
[22] "fls2_0h_Maya2.2"     "fls2_1h_Maya2"       "fls2_1h_Maya2.1"    
[25] "fls2_1h_Maya2.2"     "fls2_2h_Maya2"       "fls2_2h_Maya2.1"    
[28] "fls2_2h_Maya2.2"     "WT_1h_flg22"         "WT_1h_flg22.1"      
[31] "WT_2h_flg22"         "WT_2h_flg22.1"       "fls2.log2FoldChange"
[34] "fls2.padj"